In [1]:
%matplotlib inline
import os
import re
import collections
import random
import math

import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F

# 1.text preprocessing

typical preprocess steps:

    1.Load text as strings into memory, with some re.sub.

    2.Split strings into tokens (e.g., words and characters).

    3.Build a table of vocabulary to map the split tokens to numerical indices.

    4.Convert text into sequences of numerical indices so they can be manipulated by models easily.
    
    5.Form data iter

## 1.1 load text

In [2]:
def read_novels():
    """
    read lines of 10 famous novels.
    basic re.
    """
    lines = []
    folder_path = "../data/novels"
    for file in os.listdir(folder_path):
        if not file.startswith("."):
            lines += open(os.path.join(folder_path, file), "r").readlines()
    lines = [re.sub('[^A-Za-z]', ' ', line).strip().lower() for line in lines]
    return [line for line in lines if line]

In [3]:
lines = read_novels()

## 1.2 tokenize

In [4]:
def tokenize(lines, token='char'):
    """
    Split text lines into word or character tokens.
    we focus on char now
    """
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('ERROR: unknown token type: ' + token)

In [5]:
tokens = tokenize(lines)

## 1.3 bulid vocabulary

The string type of the token is inconvenient to be used by models, which take numerical inputs. 

Now let us build a dictionary, often called vocabulary as well, to map string tokens into numerical indices starting from 0

In [6]:
def count_corpus(tokens):
    """
    Count token frequencies.
    """
    return collections.Counter([token for line in tokens for token in line])

In [7]:
class Vocab:
    """
    Vocabulary for text.
    """
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # Sort according to frequencies
        counter = count_corpus(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                  reverse=True)
        # The index for the unknown token is 0
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens
        uniq_tokens += [token for token, freq in self.token_freqs
                        if freq >= min_freq and token not in uniq_tokens]
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        """
        tokens may be list, tuple, string
        """
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        """
        indexs may be list, tuple, int 
        """
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

In [8]:
vocab = Vocab(tokens)

## 1.4 putting all things together

In [9]:
def load_corpus_novels():
    """
    Return token indices and the vocabulary of the novels dataset.
    """
    lines = read_novels()
    tokens = tokenize(lines)
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line if vocab[token] != 0]
    return corpus, vocab

In [10]:
corpus, vocab = load_corpus_novels()

In [11]:
len(corpus), len(vocab)

(410043, 28)

## 1.5 form data iter

In [12]:
def seq_data_iter(corpus, batch_size, num_steps):
    """
    Generate a minibatch
    result shape: (batch_size, num_steps)
    """
    # Start with a random offset to partition a sequence
    offset = random.randint(0, num_steps)
    num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
    Xs = torch.tensor(corpus[offset: offset + num_tokens]).reshape(batch_size, -1)
    Ys = torch.tensor(corpus[offset + 1: offset + 1 + num_tokens]).reshape(batch_size, -1)
    
    num_batches = Xs.shape[1] // num_steps
    for i in range(0, num_steps * num_batches, num_steps):
        X = Xs[:, i: i + num_steps]
        Y = Ys[:, i: i + num_steps]
        yield X, Y

In [13]:
class SeqDataLoader:
    """
    An iterator to load sequence data.
    """
    def __init__(self, batch_size, num_steps):
        self.corpus, self.vocab = load_corpus_novels()
        self.batch_size, self.num_steps = batch_size, num_steps
        self.data_iter_fn = seq_data_iter

    def __iter__(self):
        return self.data_iter_fn(self.corpus, self.batch_size, self.num_steps)

In [14]:
def load_data_novels(batch_size, num_steps):
    """
    Return the iterator and the vocabulary of the novel dataset.
    """
    data_iter = SeqDataLoader(batch_size, num_steps)
    return data_iter, data_iter.vocab

In [15]:
batch_size, num_steps = 32, 40
train_iter, vocab = load_data_novels(batch_size, num_steps)

# 2.basic language model

assume that the tokens in a text sequence of length  $T$  are in turn  $x_{1},x_{2},...,x_{T}$

the goal of language model is to estimate the joint-probability of the sequence:

$$P(x_{1},p_{2},...,x_{T})$$

an ideal language model would be able to generate natural text just on its own, simply by drawing one token at a time:

$$x_{t} \sim P(x_{t}|x_{1},...,x_{t-1})$$

## 2.1 n-gram

how to compute the joint-probabilty? let us start by applying basic probability rules:

$$P(x_{1},...,x_{T}) = \prod_{t=1}^{T}P(x_{t}|x_{1},...,x_{t-1})$$

probability could be estimated by frequency, for example:

$$\hat{P}(learning|deep) = \frac{n(deep, learning)}{n(deep)}$$

addtionally, we commonly perform laplace smoothing:

$$\hat{P(x)} = \frac{n(x) + \epsilon_{1}/m}{n + \epsilon_{1}}$$

$$\hat{P(x'|x)}=\frac{n(x,x') + \epsilon_{2}p(x')}{n(x) + \epsilon_{2}}$$

$$\hat{P(x''|x,x')}=\frac{n(x, x', x'') + \epsilon_{3}p(x'')}{n(x, x') + \epsilon_{3}}$$

with markov assumption, we get n-gram:

$$P(x_{t}|x_{1},...,x_{t-1}) = P(x_{t}|x_{t-n+1},...,x_{t-1})$$

so for unigram, bigram, trigram, we each have:

$$P(x_{1}, x_{2}, x_{3}, x_{4}) = P(x_{1})P(x_{2})P(x_{3})p(x_{4})$$

$$P(x_{1}, x_{2}, x_{3}, x_{4}) = P(x_{1})P(x_{2}|x_{1})P(x_{3}|x_{2})p(x_{4}|x_{3})$$

$$P(x_{1}, x_{2}, x_{3}, x_{4}) = P(x_{1})P(x_{2}|x_{1})P(x_{3}|x_{1},x_{2})p(x_{4}|x_{2},x_{3})$$

## 2.2 perplexity

how to measure the language model quality?

we can ask about predicting the next token given the current set of tokens. A better language model should allow us to predict the next token more accurately

thus can measure it by the cross-entropy loss averaged over all the  $n$  tokens of a sequence:

$$\frac{1}{n}\sum_{t=1}^{n}-log\ P(x_{t}|x_{t-1},...,x_{1})$$

where  𝑃  is given by a language model and  $x_{t}$  is the actual token observed at time step  $t$  from the sequence. 

This makes the performance on documents of different lengths comparable.

For historical reasons, we use exponentials called perplexity:

$$exp\left (\frac{1}{n}\sum_{t=1}^{n}-log\ P(x_{t}|x_{t-1},...,x_{1})\right )$$

# 3. rnn model

in n-gram models, the conditional probability of a word $x_{t}$ only depends on the $n - 1$ previous words.

if we want to incorporate the possible effect of words earlier than time step $t - (n - 1)$, we need to increase $n$
. but the number of model parameters $\left | V \right | ^{n}$ increase exponentially

Hence, rather than modeling  $P(x_{t}|x_{t-n+1},...,x_{t-1})$,  it is preferable to use a latent variable model:

$$P(x_{t}|x_{1},...,x_{t-1}) \approx P(x_{t}|h_{t-1})$$

normal mlp with one hidden layer:

$$H = \phi(XW_{xh} + b_{h})$$

$$O = HW_{hq} + b_{q}$$

for sequence problem, Assume that we have a minibatch of inputs  $X_{t}\in \mathbb{R}^{n\times{d}}$  at time step $t$, then use previous latent variable:

$$H_{t} = \phi(X_{t}W_{xh} + H_{t-1}W_{hh} + b_{h})$$

$$O_{t} = H_{t}W_{hq} + b_{q}$$

often $\phi = tanh$ as default.

RNN-based Character-Level Language Models:

![jupyter](./rnn-train.svg)

## 3.1 build model

In [16]:
num_hiddens = 256
rnn_layer = nn.RNN(len(vocab), num_hiddens)

nn.RNN descrption:

For each element in the input sequence, each layer computes the following function:

$$h_t = \tanh(W_{ih} x_t + b_{ih} + W_{hh} h_{(t-1)} + b_{hh})$$

here $h_t$ is the hidden state at time $t$.<br>
$x_t$ is the input at time $t$.<br>
$h_{(t-1)}$ is the hidden state of the previous layer at time $t-1$ or the initial hidden state at time $0$.

In [17]:
class RNNModel(nn.Module):
    """
    The RNN model.
    """
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        # (batch_size, num_steps) to (num_steps, batch_size, vocab_size)
        X = F.one_hot(inputs.T.long(), self.vocab_size).type(torch.float32)
        Y, state = self.rnn(X, state)
        # The fully connected layer will first change the shape of `Y` to
        # (`num_steps` * `batch_size`, `num_hiddens`). Its output shape is
        # (`num_steps` * `batch_size`, `vocab_size`).
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            return torch.zeros((self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens))
        else:
            # lstm has two states
            return (torch.zeros((self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens)),
                   torch.zeros((self.num_directions * self.rnn.num_layers, batch_size, self.num_hiddens)))

In [18]:
net = RNNModel(rnn_layer, vocab_size=len(vocab))

## 3.2 predict & train functions

predict function

In [19]:
def predict_ch8(prefix, num_preds, net, vocab):
    """
    Generate new characters following the prefix.
    """
    state = net.begin_state(batch_size=1) # for single predict, use batch_size=1, net need to implement
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.reshape(torch.tensor([outputs[-1]]), (1, 1)) # batch_size, num_steps = 1, 1
    # warm-up period
    for y in prefix[1:]:
        _, state = net(get_input(), state) # call of net
        outputs.append(vocab[y])
    # predict num_preds steps
    for _ in range(num_preds):
        y, state = net(get_input(), state)
        # y shape: (1*1, vocab_size)
        outputs.append(int(y.argmax(dim=1))) # output max-probability
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [20]:
predict_ch8('time traveller ', 10, net, vocab)

'time traveller cccicccicc'

train epoch function

In [21]:
def train_epoch_ch8(net, train_iter, loss, updater):
    """
    Train a net within one epoch
    """
    state = None
    metric = [0.0, 0.0] # Sum of training loss, no. of tokens
    for X, Y in train_iter: # shape: (batch_size, num_steps)
        if state is None:
            # Initialize state when it is the first iteration
            state = net.begin_state(batch_size=X.shape[0])
        else:
            if isinstance(state, tuple): # lstm has two states
                for s in state:
                    s.detach_()
            else:
                state.detach_()
        y = Y.T.reshape(-1) # shape: batch_size * num_steps
        y_hat, state = net(X, state) # y_hat shape: (num_steps * batch_size, vocab_size)
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # Since the mean function has been invoked
            updater(batch_size=1)
        metric[0] += float(l * len(y))
        metric[1] += len(y)
        # print(metric)
    print(metric)
    return metric[0] / metric[1] # perplexity

train function

In [22]:
def train_ch8(net, train_iter, vocab, lr, num_epochs):
    """
    Train a model
    """
    loss = nn.CrossEntropyLoss()
    # Initialize
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, 30, net, vocab)
    # Train and predict
    for epoch in range(num_epochs):
        ppl = train_epoch_ch8(net, train_iter, loss, updater)
        if (epoch + 1) % 10 == 0:
            print(f"epoch: {epoch + 1}, log ppl: {ppl}, predict: {predict('time traveller')}")
    print(f'log perplexity {ppl:.1f}')
    print(predict('time traveller'))
    print(predict('traveller'))

In [23]:
num_epochs, lr = 20, 0.1
train_ch8(net, train_iter, vocab, lr, num_epochs)

[1134688.7014160156, 409600.0]
[1053422.9528808594, 409600.0]
[998106.9992675781, 409600.0]
[968259.9526367188, 409600.0]
[949861.7319335938, 409600.0]
[936676.0495605469, 409600.0]
[926322.6127929688, 409600.0]
[917372.7902832031, 409600.0]
[908847.6594238281, 409600.0]
[900407.0422363281, 409600.0]
epoch: 10, log ppl: 2.1982593804597856, predict: time traveller  and the  and the  and the  a
[891629.5231933594, 409600.0]
[882916.85546875, 409600.0]
[874445.2873535156, 409600.0]
[866358.7131347656, 409600.0]
[859273.8146972656, 409600.0]
[851806.4616699219, 409600.0]
[845269.6843261719, 409600.0]
[839258.4560546875, 409600.0]
[832787.9577636719, 409600.0]
[826970.4899902344, 409600.0]
epoch: 20, log ppl: 2.018970922827721, predict: time traveller  and the was the was the was 
log perplexity 2.0
time traveller  and the was the was the was 
traveller  and the was the was the was 


# 4.GRU

when calculate gradient in RNNs, long products of matrices can lead to vanishing or exploding gradients.

A number of methods have been proposed to address this, one is gru.

The key distinction between RNNs and GRUs is that the latter support gating of the hidden state

This means that we have mechanisms for when a hidden state should be updated and also when it should be reset

For instance, if the first token is of great importance we will learn not to update the hidden state after the first observation. Likewise, we will learn to skip irrelevant temporary observations

## 4.1 reset gate and update gate:

![jupyter](./gru-1.svg)

equations:

$$
\begin{aligned}
R_t = \sigma(X_t W_{xr} + H_{t-1} W_{hr} + b_r),\\
Z_t = \sigma(X_t W_{xz} + H_{t-1} W_{hz} + b_z),
\end{aligned}
$$

where $X_{t} \in \mathbb{R}^{n \times d}, W_{xr},W_{xz} \in \mathbb{R}^{d \times h}, H_{t-1} \in \mathbb{R}^{n \times h}, W_{hr},W_{hz} \in \mathbb{R}^{h \times h}, b_r,b_z \in \mathbb{R}^{1 \times h}$.

finally $R_t, Z_t \in \mathbb{R}^{n \times h}$ with elements in $(0, 1)$.

## 4.2 candidate hidden state

integrate the reset gate  $R_{t}$  with the regular latent state updating:

$$\tilde{H}_{t} = tanh(X_{t}W_{xh} + (R_{t} \odot H_{t-1})W_{hh} + b_{h})$$

the difference between RNNs and GRUs:

$$H_{t-1} \to R_{t} \odot H_{t-1}$$

reset gate: control how much of the previous state we might still want to remember, better capture short dependencies.

RNNs: how much to remember is fixed along all steps. 

GRUs: how much to remember is flexible determined by $(X_{t}, H_{t-1})$

## 4.3 hidden state

incorporate the effect of the update gate  $Z_{t}$. 

$$H_{t} = Z_{t} \odot H_{t-1} + (1 - Z_{t}) \odot \tilde{H}_{t}$$

$Z_{t}$  is close to 1: we simply retain the old state, In this case the information from  $X_{t}$  is essentially ignored.

$Z_{t}$ is close to 0: $H_{t}$  approaches the candidate latent state  $\tilde{H}_{t}$

this design: better capture dependencies for sequences with large time step distances.

![jupyter](./gru-3.svg)

reset gate $R_{t}$: inner, short dependencies.

update gate $Z_{t}$: outer, long dependencies.

## 4.4 gru implementation

In [24]:
gru_layer = nn.GRU(len(vocab), num_hiddens)
gru_net = RNNModel(gru_layer, vocab_size=len(vocab))

In [25]:
num_epochs, lr = 20, 1
train_ch8(gru_net, train_iter, vocab, lr, num_epochs)

[1054971.4916992188, 409600.0]
[944450.4013671875, 409600.0]
[901697.2551269531, 409600.0]
[863771.1938476562, 409600.0]
[832967.2651367188, 409600.0]
[807706.5061035156, 409600.0]
[785972.5317382812, 409600.0]
[766509.771484375, 409600.0]
[748810.5920410156, 409600.0]
[732009.4262695312, 409600.0]
epoch: 10, log ppl: 1.7871323883533479, predict: time traveller  and the some   said the cour
[717385.9912109375, 409600.0]
[703166.6278076172, 409600.0]
[690413.8989257812, 409600.0]
[678581.875, 409600.0]
[666910.7310791016, 409600.0]
[656641.9514160156, 409600.0]
[647003.2758789062, 409600.0]
[637546.5866699219, 409600.0]
[628813.0640869141, 409600.0]
[621071.2762451172, 409600.0]
epoch: 20, log ppl: 1.5162872955203057, predict: time traveller  and the look of the look of 
log perplexity 1.5
time traveller  and the look of the look of 
traveller  and the look of the look of 


# 5. LSTM

challenges: long-term information preservation and short-term input skipping.

## 5.1 input gate, forget gate, output gate

equations:

$$
\begin{aligned}
I_t = \sigma(X_t W_{xi} + H_{t-1} W_{hi} + b_i),\\
F_t = \sigma(X_t W_{xf} + H_{t-1} W_{hf} + b_f),\\
O_t = \sigma(X_t W_{xo} + H_{t-1} W_{ho} + b_o)
\end{aligned}
$$

where $X_{t} \in \mathbb{R}^{n \times d}, W_{xi},W_{xf},W_{xo} \in \mathbb{R}^{d \times h}, H_{t-1} \in \mathbb{R}^{n \times h}, W_{hi},W_{hf},W_{ho} \in \mathbb{R}^{h \times h}, b_i,b_f,b_o \in \mathbb{R}^{1 \times h}$.

finally $I_t, F_t, O_t \in \mathbb{R}^{n \times h}$ with elements in $(0, 1)$.

## 5.2 memory cell

we first have the candidate memory cell using the tanh activation:

$$\tilde{C}_{t} = tanh(X_t W_{xc} + H_{t-1} W_{hc} + b_c)$$

![jupyter](./lstm-1.svg)

forget gate $F_{t}$: addresses how much of the old memory cell content  $C_{t-1}$  we retain.

input gate $I_{t}$: how much we take new data into account via  $\tilde{C}_{t}$

$$C_{t} = F_{t} \odot C_{t-1} + I_{t} \odot \tilde{C}_{t}$$

## 5.3 hidden state

output gate: how to use memory in hidden state.

$$H_{t} = O_{t} \odot tanh(C_{t})$$

$H_{t}$ is therefore in the interval $(-1, 1)$.

![jupyter](./lstm-3.svg)

output gate $O_{t}$: outer, long term dependencies.

input gate $I_{t}$, forget gate $F_{t}$: inner, short term dependencies.

## 5.4 lstm implementation

In [26]:
lstm_layer = nn.LSTM(len(vocab), num_hiddens)
lstm_net = RNNModel(lstm_layer, vocab_size=len(vocab))

In [27]:
num_epochs, lr = 20, 1
train_ch8(lstm_net, train_iter, vocab, lr, num_epochs)

[1118021.6059570312, 409600.0]
[988276.4377441406, 409600.0]
[926200.8269042969, 409600.0]
[878798.1450195312, 409600.0]
[845311.0756835938, 409600.0]
[819399.6647949219, 409600.0]
[797496.4208984375, 409600.0]
[779348.2241210938, 409600.0]
[763585.3178710938, 409600.0]
[749472.0363769531, 409600.0]
epoch: 10, log ppl: 1.8297657138109207, predict: time traveller  and the was she was so the w
[736294.2172851562, 409600.0]
[724202.1860351562, 409600.0]
[713321.5804443359, 409600.0]
[702926.9617919922, 409600.0]
[692832.9311523438, 409600.0]
[682982.3431396484, 409600.0]
[674459.2429199219, 409600.0]
[665829.2003173828, 409600.0]
[657729.9713134766, 409600.0]
[650052.0603027344, 409600.0]
epoch: 20, log ppl: 1.5870411628484726, predict: time traveller  and she was not the look  an
log perplexity 1.6
time traveller  and she was not the look  an
traveller  and she was not the look  an


# 6. Deep RNN

![jupyter](./deep-rnn.svg)

as above:

$$H_{t}^{(l)}=\phi_{l}(H_{t}^{(l-1)}W_{xh}^{(l)} + H_{t-1}^{(l)}W_{hh}^{(l)} + b_{h}^{(l)})$$

for $l=1,...,L$

$$O_{t} = H_{t}^{(L)}W_{hq} + b_{q}$$

In [28]:
num_layers = 2
deep_lstm_layer = nn.LSTM(len(vocab), num_hiddens, num_layers)
deep_net = RNNModel(deep_lstm_layer, len(vocab))

In [29]:
num_epochs, lr = 20, 1
train_ch8(deep_net, train_iter, vocab, lr, num_epochs)

[1150702.6823730469, 409600.0]
[1133167.1555175781, 409600.0]
[1078806.6975097656, 409600.0]
[993128.4448242188, 409600.0]
[942791.634765625, 409600.0]
[899306.4836425781, 409600.0]
[865599.5139160156, 409600.0]
[837706.634765625, 409600.0]
[813521.7421875, 409600.0]
[791230.4956054688, 409600.0]
epoch: 10, log ppl: 1.931715077161789, predict: time traveller  and the was the was the was 
[770275.5971679688, 409600.0]
[750647.5651855469, 409600.0]
[731986.6030273438, 409600.0]
[714455.7307128906, 409600.0]
[698431.6827392578, 409600.0]
[682560.5623779297, 409600.0]
[668317.4476318359, 409600.0]
[654679.9007568359, 409600.0]
[642562.7067871094, 409600.0]
[631049.6389160156, 409600.0]
epoch: 20, log ppl: 1.5406485325098038, predict: time traveller  and the word  and the word  
log perplexity 1.5
time traveller  and the word  and the word  
traveller  and the word  and the word  


# 7. bidirectional

![jupyter](./birnn.svg)

as above:

$$\overrightarrow{H}_{t} = \phi(X_{t}W_{xh}^{(f)} + \overrightarrow{H}_{t-1}W_{hh}^{(f)} + b_{h}^{(f)})$$

$$\overleftarrow{H}_{t} = \phi(X_{t}W_{xh}^{(b)} + \overleftarrow{H}_{t-1}W_{hh}^{(b)} + b_{h}^{(b)})$$

we concatenate $\overrightarrow{H}_{t}$ adn $\overleftarrow{H}_{t}$ to obtain $H_{t} \in \mathbb{R}^{n \times 2h}$, output obtained as before:

$$O_{t} = H_{t}W_{hq} + b_{q}$$

In [30]:
num_layers = 2
bidirect_lstm_layer = nn.LSTM(len(vocab), num_hiddens, num_layers, bidirectional=True)
bidirect_net = RNNModel(deep_lstm_layer, len(vocab))

In [31]:
num_epochs, lr = 20, 1
train_ch8(bidirect_net, train_iter, vocab, lr, num_epochs)

[709594.7413330078, 409600.0]
[634474.9022216797, 409600.0]
[617450.8214111328, 409600.0]
[605551.7904052734, 409600.0]
[595406.4786376953, 409600.0]
[586651.1726074219, 409600.0]
[578493.7227783203, 409600.0]
[570757.8781738281, 409600.0]
[563096.3681640625, 409600.0]
[557130.7687988281, 409600.0]
epoch: 10, log ppl: 1.360182541012764, predict: time traveller  and the subser  and the subs
[549139.5981445312, 409600.0]
[543127.6768798828, 409600.0]
[536264.2211914062, 409600.0]
[529680.9241943359, 409600.0]
[523197.787109375, 409600.0]
[517423.6015625, 409600.0]
[510785.7235107422, 409600.0]
[504198.08142089844, 409600.0]
[499437.68017578125, 409600.0]
[494444.2214355469, 409600.0]
epoch: 20, log ppl: 1.207139212489128, predict: time travellers  and the mouse was so the ro
log perplexity 1.2
time travellers  and the mouse was so the ro
traveller  and the mouse was sometimes 
